In [1]:
import numpy as np
import re
from collections import Counter

In [2]:
setnm = list(np.load('data//numpy//setnm.npy'))
idnum = list(np.load('data//numpy//idnum.npy'))
names = list(np.load('data//numpy//names.npy'))
mcost = list(np.load('data//numpy//mcost.npy'))
rarty = list(np.load('data//numpy//rarty.npy'))
types = list(np.load('data//numpy//types.npy'))
subtp = list(np.load('data//numpy//subtp.npy'))
ctext = list(np.load('data//numpy//ctext_orig.npy'))
power = list(np.load('data//numpy//power.npy'))
tough = list(np.load('data//numpy//tough.npy'))

### encode the rarities as capital letters

In [3]:
rletr = [w[0] for w in rarty]
list(set(rletr))

['S', 'C', 'R', 'M', 'U']

### encode the types, subtypes as lower-cased words separated by a symbol '·'

In [4]:
typss = []
for x in types:
    item = ''
    lst = str(x).split(' ')
    for idx, tp in enumerate(lst):
        item += tp.lower()
        if idx < len(lst)-1:
            item += '·'
    
    typss.append(item)    

In [5]:
set(typss)

{'artifact',
 'artifact·creature',
 'creature',
 'enchantment',
 'enchantment·artifact',
 'enchantment·creature',
 'instant',
 'planeswalker',
 'sorcery',
 'tribal·artifact',
 'tribal·enchantment',
 'tribal·instant',
 'tribal·sorcery'}

In [6]:
subts = []
for lst in subtp:
    item = ''
    lst = list(lst)
    for idx, subtype in enumerate(lst):
        if subtype != 'PAD':
            item += subtype.lower()
        if idx < len(lst)-1:
            if lst[idx+1] != 'PAD':
                item += '·'
    
    if item == '':
        item = '⌧'
    
    subts.append(item)    

### remove reminder text
strip all text between parentheses

In [7]:
ctext[:10]

['first strike (this creature deals combat damage before creatures without first strike.)|when Ⓝ enters the battlefield, you gain 1 life for each card in your graveyard.',
 'flying|when Ⓝ enters the battlefield, you gain 3 life.',
 "target creature gets +3/+3 and gains flying until end of turn. (it can't be blocked except by creatures with flying or reach.)",
 'whenever a creature enters the battlefield under your control, you gain life equal to its toughness.',
 "defender (this creature can't attack.)|flying",
 'artifact and enchantment spells your opponents cast cost ② more to cast.|sacrifice Ⓝ: destroy target artifact or enchantment.',
 "flying (this creature can't be blocked except by creatures with flying or reach.)|when Ⓝ enters the battlefield, destroy target enchantment.",
 'ⓍⓌ, ↷: Ⓝ deals x damage to target attacking or blocking creature.',
 'prevent the next 1 damage that would be dealt to target creature or player this turn.|draw a card.',
 "double target player's life total

In [8]:
ctext_new = []
for text in ctext:
    new_text = re.sub(r'\s\([\S\s]+\)', '', text)
    new_text = new_text.replace('|', '·')
    ctext_new.append(new_text)

In [9]:
ctext_new[:10]

['first strike·when Ⓝ enters the battlefield, you gain 1 life for each card in your graveyard.',
 'flying·when Ⓝ enters the battlefield, you gain 3 life.',
 'target creature gets +3/+3 and gains flying until end of turn.',
 'whenever a creature enters the battlefield under your control, you gain life equal to its toughness.',
 'defender·flying',
 'artifact and enchantment spells your opponents cast cost ② more to cast.·sacrifice Ⓝ: destroy target artifact or enchantment.',
 'flying·when Ⓝ enters the battlefield, destroy target enchantment.',
 'ⓍⓌ, ↷: Ⓝ deals x damage to target attacking or blocking creature.',
 'prevent the next 1 damage that would be dealt to target creature or player this turn.·draw a card.',
 "double target player's life total. shuffle Ⓝ into its owner's library."]

### combine items into string with '|' separator
[mcost|rletr|typss|subts|ctext_new|power|tough]
don't add power/tough if not needed
use START symbol: Ⓢ and EOF symbol: Ⓔ

In [10]:
cardtext = []
for i in range(len(names)):
    lst = [names[i], mcost[i], rletr[i], typss[i], subts[i], ctext_new[i]]
    
    if power[i] != '⌧' and tough[i] != '⌧':
        lst += [power[i], tough[i]]
    
    txt = '|'.join(lst)
    txt += 'Ⓔ'
    cardtext.append(txt)

In [11]:
for i in range(10):
    print(cardtext[i])

ancestor's chosen|⑤ⓌⓌ|U|creature|human·cleric|first strike·when Ⓝ enters the battlefield, you gain 1 life for each card in your graveyard.|4|4Ⓔ
angel of mercy|④Ⓦ|U|creature|angel|flying·when Ⓝ enters the battlefield, you gain 3 life.|3|3Ⓔ
angelic blessing|②Ⓦ|C|sorcery|⌧|target creature gets +3/+3 and gains flying until end of turn.Ⓔ
angelic chorus|③ⓌⓌ|R|enchantment|⌧|whenever a creature enters the battlefield under your control, you gain life equal to its toughness.Ⓔ
angelic wall|①Ⓦ|C|creature|wall|defender·flying|0|4Ⓔ
aura of silence|①ⓌⓌ|U|enchantment|⌧|artifact and enchantment spells your opponents cast cost ② more to cast.·sacrifice Ⓝ: destroy target artifact or enchantment.Ⓔ
aven cloudchaser|③Ⓦ|C|creature|bird·soldier|flying·when Ⓝ enters the battlefield, destroy target enchantment.|2|2Ⓔ
ballista squad|③Ⓦ|U|creature|human·rebel|ⓍⓌ, ↷: Ⓝ deals x damage to target attacking or blocking creature.|2|2Ⓔ
bandage|Ⓦ|C|instant|⌧|prevent the next 1 damage that would be dealt to target creatur

### save cardtexts

In [ ]:
np.save('data/card_texts.npy', cardtext)

### analysis & testing

In [12]:
txtlens = [len(s) for s in cardtext]

In [13]:
lencnts = Counter(txtlens)
zips = lencnts.most_common()
x = sorted(zips, key = lambda x: x[0], reverse=True)

In [14]:
# x

In [15]:
idx = 250
over = sum([t[1] for t in x if t[0] > idx])
undr = sum([t[1] for t in x if t[0] <= idx])
print("% of cards over chosen idx:", over/(over+undr))

% of cards over chosen idx: 0.061505434967799096


In [16]:
print(cardtext[txtlens.index(max(txtlens))])

dance of the dead|①Ⓑ|U|enchantment|aura|enchant creature card in a graveyard·when Ⓝ enters the battlefield, if it's on the battlefield, it loses "enchant creature card in a graveyard" and gains "enchant creature put onto the battlefield with Ⓝ." put enchanted creature card onto the battlefield tapped under your control and attach Ⓝ to it. when Ⓝ leaves the battlefield, that creature's controller sacrifices it.·enchanted creature gets +1/+1 and doesn't untap during its controller's untap step.·at the beginning of the upkeep of enchanted creature's controller, that player may pay ①Ⓑ. if he or she does, untap that creature.Ⓔ


## generate batches

START: Ⓢ  PAD: ⎕

In [17]:
# first get vocabulary
vocab = []
vocabset = ['⎕'] # zero-pad PAD character
for card in cardtext:
    lst = list(card)
    vocab += lst

# add vocab according to frequency
vocabset += [x[0] for x in sorted([t for t in Counter(vocab).most_common()], key=lambda x: x[1], reverse=True)]
vocabset.append('Ⓢ') # START-PAD character

c2i = dict([(c, i) for i, c in enumerate(vocabset)])
i2c = dict([(i, c) for i, c in enumerate(vocabset)])

In [18]:
c2i[' '], i2c[0]

(1, '⎕')

In [19]:
# save
np.save('c2i.npy', c2i)
np.save('i2c.npy', i2c)

In [21]:
def cardGenerator(cardtext, windowsize, batchsize, strt='Ⓢ', pad='⎕', c2i=c2i):
    
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]
    
    # for each card...
    # todo: shuffle this??
    while True:

        # new card, get index
        thiscard = []
        
        # start-pad the window
        for j in range(windowsize):    
            thiscard.append(strt)

        # add the cardtext
        thiscard += cardtext[idx]

        # pad or truncate
#         if len(thiscard) > batchsize+windowsize:
#             thiscard = thiscard[:batchsize+windowsize]
#         else:
#             while len(thiscard) < batchsize+windowsize:
#                 thiscard.append(pad)

        # test: print
        # yield(''.join(thiscard))
        
        # int-index
        thiscard = [c2i[c] for c in thiscard]
        
        # create matrix
        x = []
        y = []
        for k in range(len(thiscard)-windowsize):
            x.append(thiscard[k:(k+windowsize)])
            y.append(thiscard[k+windowsize])
        
        # reshape for sparse_categorical_crossentropy
        y = np.array(y)
        y = y[:, np.newaxis]
        
        yield(np.asarray(x), y)
        
        if i + 1 >= len(cardtext):
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]

In [22]:
itr = cardGenerator(cardtext, 4, 250)

In [23]:
# import time
# start = time.time()
# for i in range(len(cardtext)):
#     x, y = next(itr)
# end = time.time()
# print(end - start)

In [24]:
x, y = next(itr)
print(x.shape, y.shape)
for z in range(len(x)):
    print(x[z], y[z])

(130, 4) (130, 1)
[96 96 96 96] [22]
[96 96 96 22] [12]
[96 96 22 12] [7]
[96 22 12  7] [7]
[22 12  7  7] [14]
[12  7  7 14] [57]
[ 7  7 14 57] [10]
[ 7 14 57 10] [15]
[14 57 10 15] [9]
[57 10 15  9] [6]
[10 15  9  6] [1]
[15  9  6  1] [5]
[9 6 1 5] [4]
[6 1 5 4] [18]
[ 1  5  4 18] [2]
[ 5  4 18  2] [5]
[ 4 18  2  5] [8]
[18  2  5  8] [40]
[ 2  5  8 40] [35]
[ 5  8 40 35] [8]
[ 8 40 35  8] [43]
[40 35  8 43] [8]
[35  8 43  8] [10]
[ 8 43  8 10] [5]
[43  8 10  5] [2]
[ 8 10  5  2] [4]
[10  5  2  4] [3]
[5 2 4 3] [13]
[ 2  4  3 13] [5]
[ 4  3 13  5] [2]
[ 3 13  5  2] [8]
[13  5  2  8] [15]
[ 5  2  8 15] [13]
[ 2  8 15 13] [20]
[ 8 15 13 20] [4]
[15 13 20  4] [6]
[13 20  4  6] [25]
[20  4  6 25] [23]
[ 4  6 25 23] [4]
[ 6 25 23  4] [5]
[25 23  4  5] [5]
[23  4  5  5] [9]
[4 5 5 9] [7]
[5 5 9 7] [5]
[5 9 7 5] [8]
[9 7 5 8] [23]
[ 7  5  8 23] [15]
[ 5  8 23 15] [2]
[ 8 23 15  2] [6]
[23 15  2  6] [2]
[15  2  6  2] [27]
[ 2  6  2 27] [2]
[ 6  2 27  2] [5]
[ 2 27  2  5] [1]
[27  2  5  1] [30]

to-do, network notes:

create window-size, pad with START tokens
add EOF token (and remove power/toughness if null?)
in network, encode name with LSTM(s) then copy state(s) to output network
(effectively a seq-to-seq)
generator needs to PAD TO FIXED BATCH SIZE (see length chosen above)
use batch-training and reset_state